In [111]:
import numpy as np
import pandas as pd
from sklearn import tree
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
import random
import math
from torch.utils.tensorboard import SummaryWriter
from matplotlib import pyplot

from pathlib import Path
import requests
import pickle
import gzip

import torch
import math
import torch.nn.functional as F
from torch import nn
from torch import optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader

import tenseal as ts 
import base64

pd.options.display.float_format = "{:,.4f}".format

In [112]:
FILENAME = "mnist.pkl.gz"
DATA_PATH = Path("./")
PATH = DATA_PATH 
with gzip.open((PATH / FILENAME).as_posix(), "rb") as f:
        ((x_train, y_train), (x_valid, y_valid), (x_test, y_test)) = pickle.load(f, encoding="latin-1")

In [113]:
#Classification Model

In [114]:
class Net2nn(nn.Module):
    def __init__(self):
        super(Net2nn, self).__init__()
        self.fc1=nn.Linear(784,200)
        self.fc2=nn.Linear(200,200)
        self.fc3=nn.Linear(200,10)
        
    def forward(self,x):
        x=F.relu(self.fc1(x))
        x=F.relu(self.fc2(x))
        x=self.fc3(x)
        return x

In [115]:
class WrappedDataLoader:
    def __init__(self, dl, func):
        self.dl = dl
        self.func = func

    def __len__(self):
        return len(self.dl)

    def __iter__(self):
        batches = iter(self.dl)
        for b in batches:
            yield (self.func(*b))

In [116]:
def train(model, train_loader, criterion, optimizer):
    model.train()
    train_loss = 0.0
    correct = 0

    for data, target in train_loader:
        output = model(data)
        loss = criterion(output, target)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        train_loss += loss.item()
        prediction = output.argmax(dim=1, keepdim=True)
        correct += prediction.eq(target.view_as(prediction)).sum().item()
        

    return train_loss / len(train_loader), correct/len(train_loader.dataset)

In [117]:
def validation(model, test_loader, criterion):
    model.eval()
    test_loss = 0.0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            output = model(data)
            
            test_loss += criterion(output, target).item()
            prediction = output.argmax(dim=1, keepdim=True)
            correct += prediction.eq(target.view_as(prediction)).sum().item()

    test_loss /= len(test_loader)
    correct /= len(test_loader.dataset)

    return (test_loss, correct)

In [118]:
# This function compares the accuracy of the main model 
# and the local model running on each node.
def compare_local_and_merged_model_performance(number_of_samples):
    accuracy_table=pd.DataFrame(data=np.zeros((number_of_samples,3)), columns=["sample", "local_ind_model", "merged_main_model"])
    for i in range (number_of_samples):
    
        test_ds = TensorDataset(x_test_dict[name_of_x_test_sets[i]], y_test_dict[name_of_y_test_sets[i]])
        test_dl = DataLoader(test_ds, batch_size=batch_size * 2)
    
        model=model_dict[name_of_models[i]]
        criterion=criterion_dict[name_of_criterions[i]]
        optimizer=optimizer_dict[name_of_optimizers[i]]
    
        individual_loss, individual_accuracy = validation(model, test_dl, criterion)
        main_loss, main_accuracy =validation(main_model, test_dl, main_criterion )
    
        accuracy_table.loc[i, "sample"]="sample "+str(i)
        accuracy_table.loc[i, "local_ind_model"] = individual_accuracy
        accuracy_table.loc[i, "merged_main_model"] = main_accuracy

    return accuracy_table

In [119]:
#Functions for Federated Averaging 

In [120]:
#Optimizers are algorithms or methods used to minimize an
# error function(loss function)or to maximize the efficiency of production. 

# This function creates a model, optimizer and loss function for each node.
def create_model_optimizer_criterion_dict(number_of_samples):
    model_dict = dict()
    optimizer_dict= dict()
    criterion_dict = dict()
    
    for i in range(number_of_samples):
        model_name="model"+str(i)
        model_info=Net2nn()
        model_dict.update({model_name : model_info })
        
        optimizer_name="optimizer"+str(i)
        optimizer_info = torch.optim.SGD(model_info.parameters(), lr=learning_rate, momentum=momentum)
        optimizer_dict.update({optimizer_name : optimizer_info })
        
        criterion_name = "criterion"+str(i)
        criterion_info = nn.CrossEntropyLoss()
        criterion_dict.update({criterion_name : criterion_info})
        
        
    return model_dict, optimizer_dict, criterion_dict

In [121]:
# This function takes the average of the weights in individual nodes.

def get_averaged_weights(model_dict, number_of_samples):
   
    fc1_mean_weight = torch.zeros(size=model_dict[name_of_models[0]].fc1.weight.shape)
    fc1_mean_bias = torch.zeros(size=model_dict[name_of_models[0]].fc1.bias.shape)
    
    fc2_mean_weight = torch.zeros(size=model_dict[name_of_models[0]].fc2.weight.shape)
    fc2_mean_bias = torch.zeros(size=model_dict[name_of_models[0]].fc2.bias.shape)
    
    fc3_mean_weight = torch.zeros(size=model_dict[name_of_models[0]].fc3.weight.shape)
    fc3_mean_bias = torch.zeros(size=model_dict[name_of_models[0]].fc3.bias.shape)
    
    with torch.no_grad():
    
    
        for i in range(number_of_samples):
            fc1_mean_weight += model_dict[name_of_models[i]].fc1.weight.data.clone()
            fc1_mean_bias += model_dict[name_of_models[i]].fc1.bias.data.clone()
        
            fc2_mean_weight += model_dict[name_of_models[i]].fc2.weight.data.clone()
            fc2_mean_bias += model_dict[name_of_models[i]].fc2.bias.data.clone()
        
            fc3_mean_weight += model_dict[name_of_models[i]].fc3.weight.data.clone()
            fc3_mean_bias += model_dict[name_of_models[i]].fc3.bias.data.clone()

        
        fc1_mean_weight =fc1_mean_weight/number_of_samples
        fc1_mean_bias = fc1_mean_bias/ number_of_samples
    
        fc2_mean_weight =fc2_mean_weight/number_of_samples
        fc2_mean_bias = fc2_mean_bias/ number_of_samples
    
        fc3_mean_weight =fc3_mean_weight/number_of_samples
        fc3_mean_bias = fc3_mean_bias/ number_of_samples
    
    return fc1_mean_weight, fc1_mean_bias, fc2_mean_weight, fc2_mean_bias, fc3_mean_weight, fc3_mean_bias

In [122]:
# This function sends the averaged weights of individual nodes 
# to the main model and sets them as the new weights of the main model. ( calls def get_averaged_weights(model_dict, number_of_samples))

def set_averaged_weights_as_main_model_weights_and_update_main_model(main_model,model_dict, number_of_samples):
    fc1_mean_weight, fc1_mean_bias, fc2_mean_weight, fc2_mean_bias, fc3_mean_weight, fc3_mean_bias = get_averaged_weights(model_dict, number_of_samples=number_of_samples)
    with torch.no_grad():
        main_model.fc1.weight.data = fc1_mean_weight.data.clone()
        main_model.fc2.weight.data = fc2_mean_weight.data.clone()
        main_model.fc3.weight.data = fc3_mean_weight.data.clone()

        main_model.fc1.bias.data = fc1_mean_bias.data.clone()
        main_model.fc2.bias.data = fc2_mean_bias.data.clone()
        main_model.fc3.bias.data = fc3_mean_bias.data.clone() 
    return main_model

In [123]:
# This function sends the parameters of the main model to the nodes.
# The following data should send from SERVER to each node
# So we should save them on a file

def send_main_model_to_nodes_and_update_model_dict(main_model,want_print):
    with torch.no_grad():
            torch.save(main_model.fc1.weight.data.clone(), '/Users/tung/Desktop/Federated-Learning-Code/Main_Model_Parameters/main_model_fc1_weight_data.pt')
            torch.save(main_model.fc2.weight.data.clone(), '/Users/tung/Desktop/Federated-Learning-Code/Main_Model_Parameters/main_model_fc2_weight_data.pt')
            torch.save(main_model.fc3.weight.data.clone(), '/Users/tung/Desktop/Federated-Learning-Code/Main_Model_Parameters/main_model_fc3_weight_data.pt')
            
            torch.save(main_model.fc1.bias.data.clone(), '/Users/tung/Desktop/Federated-Learning-Code/Main_Model_Parameters/main_model_fc1_bias_data.pt')
            torch.save(main_model.fc2.bias.data.clone(), '/Users/tung/Desktop/Federated-Learning-Code/Main_Model_Parameters/main_model_fc2_bias_data.pt')
            torch.save(main_model.fc3.bias.data.clone(), '/Users/tung/Desktop/Federated-Learning-Code/Main_Model_Parameters/main_model_fc3_bias_data.pt')

            if (want_print == 1):
                print(main_model.fc1.weight.data.clone())
                print("--------------------------------------------------------")
                print(main_model.fc2.weight.data.clone())
                print("--------------------------------------------------------")
                print(main_model.fc3.weight.data.clone())
                print("--------------------------------------------------------")

                print(main_model.fc1.bias.data.clone())
                print("--------------------------------------------------------")
                print(main_model.fc2.bias.data.clone())
                print("--------------------------------------------------------")
                print(main_model.fc3.bias.data.clone()) 



In [124]:
# This function downloads all of the local models

def download_local_model_from_each_node(number_of_samples):
    with torch.no_grad():
        for i in range(number_of_samples):
            model_dict[name_of_models[i]].fc1.weight.data = torch.load('/Users/tung/Desktop/Federated-Learning-Code/Local_Model_Parameters/local_model_'+str(i)+'_fc1_weight_data.pt')
            model_dict[name_of_models[i]].fc2.weight.data = torch.load('/Users/tung/Desktop/Federated-Learning-Code/Local_Model_Parameters/local_model_'+str(i)+'_fc2_weight_data.pt')
            model_dict[name_of_models[i]].fc3.weight.data = torch.load('/Users/tung/Desktop/Federated-Learning-Code/Local_Model_Parameters/local_model_'+str(i)+'_fc3_weight_data.pt')
            
            model_dict[name_of_models[i]].fc1.bias.data = torch.load('/Users/tung/Desktop/Federated-Learning-Code/Local_Model_Parameters/local_model_'+str(i)+'_fc1_bias_data.pt')
            model_dict[name_of_models[i]].fc2.bias.data = torch.load('/Users/tung/Desktop/Federated-Learning-Code/Local_Model_Parameters/local_model_'+str(i)+'_fc2_bias_data.pt')
            model_dict[name_of_models[i]].fc3.bias.data = torch.load('/Users/tung/Desktop/Federated-Learning-Code/Local_Model_Parameters/local_model_'+str(i)+'_fc3_bias_data.pt')
    
    return model_dict

In [125]:
x_train, y_train, x_valid, y_valid,x_test, y_test = map(torch.tensor, (x_train, y_train, x_valid, y_valid, x_test, y_test))
number_of_samples = 3
learning_rate = 0.01
numEpoch = 10
batch_size = 32
momentum = 0.9

train_amount = 4500
valid_amount = 900
test_amount = 900
print_amount = 3

In [126]:
# Main model is created


In [127]:
main_model = Net2nn()
main_optimizer = torch.optim.SGD(main_model.parameters(), lr=learning_rate, momentum=0.9)
main_criterion = nn.CrossEntropyLoss()

In [128]:
print(main_model.fc2.weight[0:1,0:5])



tensor([[-0.0455,  0.0371,  0.0559,  0.0530,  0.0350]],
       grad_fn=<SliceBackward0>)


In [129]:
# The following data should send from SERVER to each node
# So we should save them on a file

send_main_model_to_nodes_and_update_model_dict(main_model,1) # if 1, then print weight and bias


tensor([[ 2.8787e-02, -1.2590e-02,  2.7635e-02,  ..., -1.8766e-02,
          1.7136e-02, -2.8596e-02],
        [-1.7997e-02, -1.3210e-02,  4.9353e-05,  ...,  2.9637e-02,
          4.9759e-03,  2.2190e-02],
        [ 2.7930e-02, -2.0363e-02, -1.3893e-03,  ...,  3.0865e-02,
          2.5569e-02,  3.4142e-02],
        ...,
        [-3.0738e-02,  2.4559e-02, -1.9661e-02,  ...,  2.8706e-02,
         -2.0387e-02, -2.4562e-02],
        [ 9.4455e-03,  1.9088e-02, -3.5383e-02,  ..., -6.8341e-03,
         -1.8190e-02, -7.6064e-04],
        [ 2.6892e-02, -1.9745e-02, -1.1494e-02,  ..., -1.0431e-02,
          3.0275e-03,  6.1734e-03]])
--------------------------------------------------------
tensor([[-0.0455,  0.0371,  0.0559,  ...,  0.0170,  0.0307,  0.0253],
        [ 0.0604,  0.0029,  0.0529,  ...,  0.0291,  0.0420, -0.0167],
        [ 0.0648, -0.0082, -0.0505,  ...,  0.0312, -0.0451,  0.0209],
        ...,
        [ 0.0205,  0.0037,  0.0689,  ..., -0.0331,  0.0560,  0.0300],
        [ 0.0378, 

In [130]:
print(main_model.fc2.weight[0:1,0:5])



tensor([[-0.0455,  0.0371,  0.0559,  0.0530,  0.0350]],
       grad_fn=<SliceBackward0>)


In [131]:
model_dict = dict()
for i in range(number_of_samples):
    model_name="model"+str(i)
    model_info=Net2nn()
    model_dict.update({model_name : model_info })

name_of_models=list(model_dict.keys())


In [132]:
# After the first time run, we should run the following code


# This function downloads all of the local models
# We should first run the client code. Then here.
# model_dict = download_local_model_from_each_node(number_of_samples)
# print(model_dict["model1"].fc2.weight[0,0:5])
# print(model_dict["model0"].fc2.weight[0,0:5])
# print("--------------------------------------------------")



# main_model = set_averaged_weights_as_main_model_weights_and_update_main_model(main_model,model_dict, number_of_samples) 

# print(main_model.fc2.weight[0:1,0:5])
# print("--------------------------------------------------")



# train_ds = TensorDataset(x_train, y_train)
# train_dl = DataLoader(train_ds, batch_size=batch_size, shuffle=True)

# valid_ds = TensorDataset(x_valid, y_valid)
# valid_dl = DataLoader(valid_ds, batch_size=batch_size * 2)

# test_ds = TensorDataset(x_test, y_test)
# test_dl = DataLoader(test_ds, batch_size=batch_size * 2)


# test_loss, test_accuracy = validation(main_model, test_dl, main_criterion)



# test_loss, test_accuracy = validation(main_model, test_dl, main_criterion)
# print("Iteration", str(i+2), ": main_model accuracy on all test data: {:7.4f}".format(test_accuracy))
# print("--------------------------------------------------")


# send_main_model_to_nodes_and_update_model_dict(main_model,0) # if 1, then print weight and bias
# For storing public and private keys
def write_data(file_name, data):
    if type(data) == bytes:
        #bytes to base64
        data = base64.b64encode(data)
         
    with open(file_name, 'wb') as f: 
        f.write(data)
 
def read_data(file_name):
    with open(file_name, "rb") as f:
        data = f.read()
    #base64 to bytes
    return base64.b64decode(data)

context = ts.context_from(read_data("public.txt"))

# Load local parameters and compute average
for i in range(200):
    # FC1
    temp = read_data("Local_Model_Parameters/encrypted_model_0_1_weight_" + str(i))
    encrypted_model_0_1_weight = ts.lazy_ckks_vector_from(temp)
    encrypted_model_0_1_weight.link_context(context)

    temp = read_data("Local_Model_Parameters/encrypted_model_1_1_weight_" + str(i))
    encrypted_model_1_1_weight = ts.lazy_ckks_vector_from(temp)
    encrypted_model_1_1_weight.link_context(context)

    temp = read_data("Local_Model_Parameters/encrypted_model_2_1_weight_" + str(i))
    encrypted_model_2_1_weight = ts.lazy_ckks_vector_from(temp)
    encrypted_model_2_1_weight.link_context(context)

    average_encrypted_model_weight = (encrypted_model_0_1_weight + encrypted_model_1_1_weight + encrypted_model_2_1_weight) * 0.333333333
    write_data('Main_Model_Parameters/encrypted_model_1_weight_' + str(i), average_encrypted_model_weight.serialize())

    # FC2
    temp = read_data("Local_Model_Parameters/encrypted_model_0_2_weight_" + str(i))
    encrypted_model_0_2_weight = ts.lazy_ckks_vector_from(temp)
    encrypted_model_0_2_weight.link_context(context)

    temp = read_data("Local_Model_Parameters/encrypted_model_1_2_weight_" + str(i))
    encrypted_model_1_2_weight = ts.lazy_ckks_vector_from(temp)
    encrypted_model_1_2_weight.link_context(context)

    temp = read_data("Local_Model_Parameters/encrypted_model_2_2_weight_" + str(i))
    encrypted_model_2_2_weight = ts.lazy_ckks_vector_from(temp)
    encrypted_model_2_2_weight.link_context(context)

    average_encrypted_model_weight = (encrypted_model_0_2_weight + encrypted_model_1_2_weight + encrypted_model_2_2_weight) * 0.333333333
    write_data('Main_Model_Parameters/encrypted_model_2_weight_' + str(i), average_encrypted_model_weight.serialize())

for i in range(10):
    # FC3
    temp = read_data("Local_Model_Parameters/encrypted_model_0_3_weight_" + str(i))
    encrypted_model_0_3_weight = ts.lazy_ckks_vector_from(temp)
    encrypted_model_0_3_weight.link_context(context)

    temp = read_data("Local_Model_Parameters/encrypted_model_1_3_weight_" + str(i))
    encrypted_model_1_3_weight = ts.lazy_ckks_vector_from(temp)
    encrypted_model_1_3_weight.link_context(context)

    temp = read_data("Local_Model_Parameters/encrypted_model_2_3_weight_" + str(i))
    encrypted_model_2_3_weight = ts.lazy_ckks_vector_from(temp)
    encrypted_model_2_3_weight.link_context(context)

    average_encrypted_model_weight = (encrypted_model_0_3_weight + encrypted_model_1_3_weight + encrypted_model_2_3_weight) * 0.333333333
    write_data('Main_Model_Parameters/encrypted_model_3_weight_' + str(i), average_encrypted_model_weight.serialize())

# Bias FC1
temp = read_data("Local_Model_Parameters/encrypted_model_0_1_bias")
encrypted_model_0_1_bias = ts.lazy_ckks_vector_from(temp)
encrypted_model_0_1_bias.link_context(context)

temp = read_data("Local_Model_Parameters/encrypted_model_1_1_bias")
encrypted_model_1_1_bias = ts.lazy_ckks_vector_from(temp)
encrypted_model_1_1_bias.link_context(context)

temp = read_data("Local_Model_Parameters/encrypted_model_2_1_bias")
encrypted_model_2_1_bias = ts.lazy_ckks_vector_from(temp)
encrypted_model_2_1_bias.link_context(context)

average_encrypted_model_bias = (encrypted_model_0_1_bias + encrypted_model_1_1_bias + encrypted_model_2_1_bias) * 0.333333333
write_data('Main_Model_Parameters/encrypted_model_1_bias', average_encrypted_model_bias.serialize())

# Bias FC2
temp = read_data("Local_Model_Parameters/encrypted_model_0_2_bias")
encrypted_model_0_2_bias = ts.lazy_ckks_vector_from(temp)
encrypted_model_0_2_bias.link_context(context)

temp = read_data("Local_Model_Parameters/encrypted_model_1_2_bias")
encrypted_model_1_2_bias = ts.lazy_ckks_vector_from(temp)
encrypted_model_1_2_bias.link_context(context)

temp = read_data("Local_Model_Parameters/encrypted_model_2_2_bias")
encrypted_model_2_2_bias = ts.lazy_ckks_vector_from(temp)
encrypted_model_2_2_bias.link_context(context)

average_encrypted_model_bias = (encrypted_model_0_2_bias + encrypted_model_1_2_bias + encrypted_model_2_2_bias) * 0.333333333
write_data('Main_Model_Parameters/encrypted_model_2_bias', average_encrypted_model_bias.serialize())

# Bias FC3
temp = read_data("Local_Model_Parameters/encrypted_model_0_3_bias")
encrypted_model_0_3_bias = ts.lazy_ckks_vector_from(temp)
encrypted_model_0_3_bias.link_context(context)

temp = read_data("Local_Model_Parameters/encrypted_model_1_3_bias")
encrypted_model_1_3_bias = ts.lazy_ckks_vector_from(temp)
encrypted_model_1_3_bias.link_context(context)

temp = read_data("Local_Model_Parameters/encrypted_model_2_3_bias")
encrypted_model_2_3_bias = ts.lazy_ckks_vector_from(temp)
encrypted_model_2_3_bias.link_context(context)

average_encrypted_model_bias = (encrypted_model_0_3_bias + encrypted_model_1_3_bias + encrypted_model_2_3_bias) * 0.333333333
write_data('Main_Model_Parameters/encrypted_model_3_bias', average_encrypted_model_bias.serialize())